Se usan los datos de la carpeta Data_football_mix.

En main están las temporadas desde 93_94 hasta 23_24.

En la carpeta de cada temporada están los archivos divididos por liga.

D1: Bundesliga

In [90]:
from pytorch_tabnet.tab_model import TabNetClassifier

import os
import torch
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
 
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [91]:
#Carpeta Main
folder = 'Data_football_mix/main'
#Carpeta de las temporadas
folder_seasons = []
for i in range(2004, 2024):
    folder_seasons.append(f'{str(i)[2:]}_{str(i+1)[2:]}')

In [92]:
#budesliga
bundesliga_seasons = []
budesliga_file = 'D1.csv'

#serie A
serie_a_seasons = []
serie_a_file = 'I1.csv'

#Ligue 1
ligue_1_seasons = []
ligue_1_file = 'F1.csv'

#La Liga
la_liga_seasons = []
la_liga_file = 'SP1.csv'

In [93]:

for season in folder_seasons:
    try:
        bundesliga_seasons.append(pd.read_csv(f'{folder}/{season}/{budesliga_file}', sep=',', encoding='Latin1'))
        serie_a_seasons.append(pd.read_csv(f'{folder}/{season}/{serie_a_file}', sep=',', encoding='Latin1'))
        ligue_1_seasons.append(pd.read_csv(f'{folder}/{season}/{ligue_1_file}', sep=',', encoding='Latin1'))
        la_liga_seasons.append(pd.read_csv(f'{folder}/{season}/{la_liga_file}', sep=',', encoding='Latin1'))
    except:
        print(f'error en la temporada {season}')

error en la temporada 04_05


In [94]:
#cada temporada es un dataframe
dfs_seasons_bundesliga = []
dfs_seasons_serie_a = []
dfs_seasons_ligue_1 = []
dfs_seasons_la_liga = []

for season in bundesliga_seasons:
    dfs_seasons_bundesliga.append(pd.DataFrame(season))

for season in serie_a_seasons:
    dfs_seasons_serie_a.append(pd.DataFrame(season))

for season in ligue_1_seasons:
    dfs_seasons_ligue_1.append(pd.DataFrame(season))

for season in la_liga_seasons:
    dfs_seasons_la_liga.append(pd.DataFrame(season))

#Nos importan las columnas Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
dfs_seasons_bundesliga = [season[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] for season in dfs_seasons_bundesliga]
dfs_seasons_serie_a = [season[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] for season in dfs_seasons_serie_a]
dfs_seasons_ligue_1 = [season[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] for season in dfs_seasons_ligue_1]
dfs_seasons_la_liga = [season[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] for season in dfs_seasons_la_liga]

dfs_seasons_serie_a[0].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,27/08/05,Fiorentina,Sampdoria,2,1,H
1,27/08/05,Livorno,Lecce,2,1,H
2,28/08/05,Ascoli,Milan,1,1,D
3,28/08/05,Inter,Treviso,3,0,H
4,28/08/05,Juventus,Chievo,1,0,H


### manejo de datos

Agregamos nuevas columnas

#### Bundesliga: 18 equipos

In [95]:
# agregar Wk a cada fila de cada temporada
# Cada equipo juega 34 partidos por temporada
leagues_18_teams = [dfs_seasons_bundesliga, dfs_seasons_ligue_1]
for league in leagues_18_teams:
    for i in range(len(league)):
        #cada vez que los 18 equipos juegan 1 partido, se aumenta el contador
        wk = 1
        counter = 1
        for j in range(len(league[i])):
            if counter == 10:
                counter = 1
                wk += 1
            league[i].loc[j, 'Wk'] = wk
            counter += 1

dfs_seasons_bundesliga[-1].head()


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Wk
0,18/08/2023,Werder Bremen,Bayern Munich,0,4,A,1.0
1,19/08/2023,Augsburg,M'gladbach,4,4,D,1.0
2,19/08/2023,Hoffenheim,Freiburg,1,2,A,1.0
3,19/08/2023,Leverkusen,RB Leipzig,3,2,H,1.0
4,19/08/2023,Stuttgart,Bochum,5,0,H,1.0


#### Resto de ligas: 20 equipos

In [96]:
leagues_20_teams = [dfs_seasons_serie_a, dfs_seasons_la_liga]

for league in leagues_20_teams:
    for i in range(len(league)):
        wk = 1
        counter = 1
        for j in range(len(league[i])):
            if counter == 11:
                counter = 1
                wk += 1
            league[i].loc[j, 'Wk'] = wk
            counter += 1

dfs_seasons_la_liga[-1].tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Wk
253,24/02/2024,Almeria,Ath Madrid,2,2,D,26.0
254,25/02/2024,Cadiz,Celta,2,2,D,26.0
255,25/02/2024,Betis,Ath Bilbao,3,1,H,26.0
256,25/02/2024,Las Palmas,Osasuna,1,1,D,26.0
257,25/02/2024,Real Madrid,Sevilla,1,0,H,26.0


In [97]:
#rename columns
#HomeTeam -> Team_home, AwayTeam -> Team_away, FTHG -> Goals_home, FTAG -> Goals_away, 
leagues = [dfs_seasons_la_liga, dfs_seasons_ligue_1, dfs_seasons_serie_a, dfs_seasons_bundesliga]
for league in leagues:
    for season in league:
        season.columns = ['Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away', 'FTR', 'Wk']
        season.drop(columns={'FTR'}, inplace=True)

In [98]:
#reodernar columnas
dfs_seasons_la_liga = [season[['Wk', 'Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away']] for season in dfs_seasons_la_liga]
dfs_seasons_ligue_1 = [season[['Wk', 'Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away']] for season in dfs_seasons_ligue_1]
dfs_seasons_serie_a = [season[['Wk', 'Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away']] for season in dfs_seasons_serie_a]
dfs_seasons_bundesliga = [season[['Wk', 'Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away']] for season in dfs_seasons_bundesliga]

dfs_seasons_ligue_1[-1].head()

,Wk,Date,Team_home,Team_away,Goals_home,Goals_away
0,1.0,11/08/2023,Nice,Lille,1,1
1,1.0,12/08/2023,Marseille,Reims,2,1
2,1.0,12/08/2023,Paris SG,Lorient,0,0
3,1.0,13/08/2023,Brest,Lens,3,2
4,1.0,13/08/2023,Clermont,Monaco,2,4


In [99]:
leagues = [dfs_seasons_la_liga, dfs_seasons_ligue_1, dfs_seasons_serie_a, dfs_seasons_bundesliga]

In [ ]:
#Agregar resultado temporada anterior


In [100]:
def remove_suffix(item):
    return item.replace("_home", "").replace("_away", "")

column = [remove_suffix(item) for item in dfs_seasons_ligue_1[0].columns]
columnas = []
[columnas.append(x) for x in column if x not in columnas]
columnas.remove("Wk")
columnas.remove("Date")
columnas.remove("Team")
columnas[0] = "score_prom"
print(columnas)
print(len(columnas))

['score_prom']
1


In [101]:
columnas.append("wins")
columnas.append("ties")
columnas.append("losses")
columnas.append("total_goals")
columnas.append("total_goals_conceded")
columnas.append("points")
columnas.append("streak")
columnas.append("wins_in_last_5_matches")
columnas.append("ties_in_last_5_matches")
columnas.append("losses_in_last_5_matches")

In [102]:
tablas = []

for dfs_seasons in leagues:
    tablas_season = []
    for torneo, i in zip(dfs_seasons, range(len(dfs_seasons))):
        equipos = torneo["Team_home"].unique()
        equipos2 = torneo["Team_away"].unique()
        stats = columnas

        tabla = {}

        for equipo in equipos:

            tabla[equipo] = dict.fromkeys(stats, 0)
            tabla[equipo]["last_5_matches"] = list()

        tablas_season.append(tabla)
    tablas.append(tablas_season)
print(stats)
print(tablas[1])

['score_prom', 'wins', 'ties', 'losses', 'total_goals', 'total_goals_conceded', 'points', 'streak', 'wins_in_last_5_matches', 'ties_in_last_5_matches', 'losses_in_last_5_matches']
[{'Paris SG': {'score_prom': 0, 'wins': 0, 'ties': 0, 'losses': 0, 'total_goals': 0, 'total_goals_conceded': 0, 'points': 0, 'streak': 0, 'wins_in_last_5_matches': 0, 'ties_in_last_5_matches': 0, 'losses_in_last_5_matches': 0, 'last_5_matches': []}, 'Lille': {'score_prom': 0, 'wins': 0, 'ties': 0, 'losses': 0, 'total_goals': 0, 'total_goals_conceded': 0, 'points': 0, 'streak': 0, 'wins_in_last_5_matches': 0, 'ties_in_last_5_matches': 0, 'losses_in_last_5_matches': 0, 'last_5_matches': []}, 'Marseille': {'score_prom': 0, 'wins': 0, 'ties': 0, 'losses': 0, 'total_goals': 0, 'total_goals_conceded': 0, 'points': 0, 'streak': 0, 'wins_in_last_5_matches': 0, 'ties_in_last_5_matches': 0, 'losses_in_last_5_matches': 0, 'last_5_matches': []}, 'Nancy': {'score_prom': 0, 'wins': 0, 'ties': 0, 'losses': 0, 'total_goals':

In [103]:
new_stats = ["wins",
"ties",
"losses",
"total_goals",
"total_goals_conceded",
"points",
"streak",
"wins_in_last_5_matches",
"ties_in_last_5_matches",
"losses_in_last_5_matches"]
procesated_matches_columns = list(dfs_seasons_ligue_1[0].columns)
procesated_matches_columns.insert(6, "score_prom_away")
procesated_matches_columns.insert(6, "score_prom_home")

for stat in new_stats:
    procesated_matches_columns.append(f"{stat}_home")
    procesated_matches_columns.append(f"{stat}_away")

procesated_matches = pd.DataFrame(columns = procesated_matches_columns)

procesated_matches.head()

,Wk,Date,Team_home,Team_away,Goals_home,Goals_away,score_prom_home,score_prom_away,wins_home,wins_away,...,points_home,points_away,streak_home,streak_away,wins_in_last_5_matches_home,wins_in_last_5_matches_away,ties_in_last_5_matches_home,ties_in_last_5_matches_away,losses_in_last_5_matches_home,losses_in_last_5_matches_away


In [104]:
dfs_seasons_ligue_1[0].columns

Index(['Wk', 'Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away'], dtype='object')

In [105]:
contador = 0
for dfs_seasons in leagues:
    for torneo, i in zip(dfs_seasons, range(len(dfs_seasons))):
        torneo.sort_values(by='Wk', inplace=True)
        tabla = tablas[contador][i]
        for index in torneo.index:
        
            """Escribir info en el nuevo df de partidos con la información hasta el partido la cual esta almacenada en 'tabla'"""
            match = torneo.loc[index]

            team_home = match["Team_home"]
            team_away = match["Team_away"]

            stats_home = tabla[team_home]
            stats_away = tabla[team_away]

            fila = []
            fila.append(match["Wk"])
            fila.append(match["Date"])
            fila.append(team_home)
            fila.append(team_away)
            fila.append(match["Goals_home"])
            fila.append(match["Goals_away"])

            for stat in stats:
                fila.append(tabla[team_home][stat])
                fila.append(tabla[team_away][stat])

            procesated_matches.loc[len(procesated_matches)] = fila
            
            
            """ Actualizar la tabla con lo que realmente ocurrió en ese partido"""

            matches_played_home = tabla[team_home]["wins"] + tabla[team_home]["ties"] + tabla[team_home]["losses"]+1
            matches_played_away = tabla[team_away]["wins"] + tabla[team_away]["ties"] + tabla[team_away]["losses"]+1
            
            ## stats promediadas:
            for stat in stats[:stats.index("wins")]:
                if stat == "score_prom":
                    tabla[team_home][stat] = (tabla[team_home][stat]*(matches_played_home-1) + match["Goals_home"])/matches_played_home
                    tabla[team_away][stat] = (tabla[team_away][stat]*(matches_played_away-1) + match["Goals_away"])/matches_played_away

            ## stats acumuladas
            
            empate = False
            if match["Goals_home"] > match["Goals_away"]:

                #victorias / derrotas
                tabla[team_home]["wins"] += 1
                tabla[team_away]["losses"] += 1

                #goles a favor
                tabla[team_home]["total_goals"] += match["Goals_home"]
                tabla[team_away]["total_goals"] += match["Goals_away"]

                #goles en contra
                tabla[team_home]["total_goals_conceded"] += match["Goals_away"]
                tabla[team_away]["total_goals_conceded"] += match["Goals_home"]

                #puntos
                tabla[team_home]["points"] += 3

                # racha
                tabla[team_home]["streak"] += 1
                tabla[team_away]["streak"] = 0

                # ultimos 5 partidos
                if len(tabla[team_home]["last_5_matches"]) == 5:
                    tabla[team_home]["last_5_matches"].pop(0)
                if len(tabla[team_away]["last_5_matches"]) == 5:
                    tabla[team_away]["last_5_matches"].pop(0)

                tabla[team_home]["last_5_matches"].append("W")
                tabla[team_away]["last_5_matches"].append("L")


            elif match["Goals_home"] < match["Goals_away"]:

                #victorias / derrotas
                tabla[team_away]["wins"] += 1
                tabla[team_home]["losses"] += 1

                #goles a favor
                tabla[team_away]["total_goals"] += match["Goals_away"]
                tabla[team_home]["total_goals"] += match["Goals_home"]

                #goles en contra
                tabla[team_away]["total_goals_conceded"] += match["Goals_home"]
                tabla[team_home]["total_goals_conceded"] += match["Goals_away"]

                #puntos
                tabla[team_away]["points"] += 3

                # racha
                tabla[team_away]["streak"] += 1
                tabla[team_home]["streak"] = 0

                # ultimos 5 partidos
                if len(tabla[team_home]["last_5_matches"]) == 5:
                    tabla[team_home]["last_5_matches"].pop(0)
                if len(tabla[team_away]["last_5_matches"]) == 5:
                    tabla[team_away]["last_5_matches"].pop(0)

                tabla[team_away]["last_5_matches"].append("W")
                tabla[team_home]["last_5_matches"].append("L")

            else:
                #empate
                tabla[team_home]["ties"] += 1
                tabla[team_away]["ties"] += 1

                #goles a favor
                tabla[team_home]["total_goals"] += match["Goals_home"]
                tabla[team_away]["total_goals"] += match["Goals_away"]

                #goles en contra
                tabla[team_home]["total_goals_conceded"] += match["Goals_away"]
                tabla[team_away]["total_goals_conceded"] += match["Goals_home"]

                #puntos
                tabla[team_home]["points"] += 1
                tabla[team_away]["points"] += 1

                # racha
                tabla[team_away]["streak"] += 1
                tabla[team_home]["streak"] += 1

                # ultimos 5 partidos
                if len(tabla[team_home]["last_5_matches"]) == 5:
                    tabla[team_home]["last_5_matches"].pop(0)
                if len(tabla[team_away]["last_5_matches"]) == 5:
                    tabla[team_away]["last_5_matches"].pop(0)

                tabla[team_away]["last_5_matches"].append("T")
                tabla[team_home]["last_5_matches"].append("T")

            tabla[team_home]["wins_in_last_5_matches"] = tabla[team_home]["last_5_matches"].count("W")
            tabla[team_home]["ties_in_last_5_matches"] = tabla[team_home]["last_5_matches"].count("T")
            tabla[team_home]["losses_in_last_5_matches"] = tabla[team_home]["last_5_matches"].count("L")

            tabla[team_away]["wins_in_last_5_matches"] = tabla[team_away]["last_5_matches"].count("W")
            tabla[team_away]["ties_in_last_5_matches"] = tabla[team_away]["last_5_matches"].count("T")
            tabla[team_away]["losses_in_last_5_matches"] = tabla[team_away]["last_5_matches"].count("L")
    
    contador += 1

In [106]:
print(len(procesated_matches))
procesated_matches.tail(500)

27168


,Wk,Date,Team_home,Team_away,Goals_home,Goals_away,score_prom_home,score_prom_away,wins_home,wins_away,...,points_home,points_away,streak_home,streak_away,wins_in_last_5_matches_home,wins_in_last_5_matches_away,ties_in_last_5_matches_home,ties_in_last_5_matches_away,losses_in_last_5_matches_home,losses_in_last_5_matches_away
26668,2.0,13/08/2022,Hertha,Ein Frankfurt,1.0,1.0,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
26669,2.0,13/08/2022,Leverkusen,Augsburg,1.0,2.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
26670,2.0,13/08/2022,Hoffenheim,Bochum,3.0,2.0,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
26671,2.0,12/08/2022,Freiburg,Dortmund,1.0,3.0,4.000000,1.000000,1.0,1.0,...,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
26672,2.0,13/08/2022,RB Leipzig,FC Koln,2.0,2.0,1.000000,3.000000,0.0,1.0,...,1.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27163,23.0,24/02/2024,Union Berlin,Heidenheim,2.0,2.0,0.954545,1.409091,7.0,7.0,...,24.0,27.0,2.0,0.0,3.0,1.0,1.0,3.0,1.0,1.0
27164,23.0,24/02/2024,Werder Bremen,Darmstadt,1.0,1.0,1.409091,1.045455,8.0,2.0,...,29.0,12.0,1.0,0.0,4.0,0.0,0.0,2.0,1.0,3.0
27165,23.0,24/02/2024,Bayern Munich,RB Leipzig,2.0,1.0,2.772727,2.181818,16.0,12.0,...,50.0,40.0,0.0,3.0,2.0,2.0,0.0,1.0,3.0,2.0
27166,23.0,25/02/2024,Ein Frankfurt,Wolfsburg,2.0,2.0,1.545455,1.181818,8.0,6.0,...,33.0,24.0,2.0,1.0,1.0,0.0,3.0,4.0,1.0,1.0


In [107]:
procesated_matches.dropna(subset=["Team_home", "Team_away"], inplace=True)

(procesated_matches[procesated_matches["Team_away"].isna()])


,Wk,Date,Team_home,Team_away,Goals_home,Goals_away,score_prom_home,score_prom_away,wins_home,wins_away,...,points_home,points_away,streak_home,streak_away,wins_in_last_5_matches_home,wins_in_last_5_matches_away,ties_in_last_5_matches_home,ties_in_last_5_matches_away,losses_in_last_5_matches_home,losses_in_last_5_matches_away


In [108]:
matches = procesated_matches

#Aqui agrega la columna de resultados

results = []

for index in matches.index:

    match = matches.loc[index]
    if match["Goals_home"] > match["Goals_away"]:
        results.append(1)
    elif match["Goals_home"] < match["Goals_away"]:
        results.append(2)
    else:
        results.append(0)

matches["result"] = results

In [109]:
#Cantidad de partidos por resultado (local, empate, visita)

for i in range(0,3):
    cantidad = matches[matches["result"]==i].shape[0]
    print(f"Resultado {i}, hay: {cantidad} partidos")
num = 722/(371+722+551)
print("azar:", num)

Resultado 0, hay: 7062 partidos
Resultado 1, hay: 12317 partidos
Resultado 2, hay: 7782 partidos
azar: 0.4391727493917275


In [110]:
#Ahora, vamos a separar el último torneo
cant_partidos_entreno = 0
for i in range (1, 6):
    cant_partidos_entreno += len(dfs_seasons_bundesliga[-i])
print(cant_partidos_entreno)

matches_train = matches.iloc[:-cant_partidos_entreno]
matches_test = matches.iloc[-cant_partidos_entreno:]

result_train = matches_train["result"]
result_test = matches_test["result"]
matches_train = matches_train.drop(columns = ["Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk", "Date"])
matches_test = matches_test.drop(columns = ["Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk", "Date"])

matches_train.tail(), matches_test.tail()


1431


(       score_prom_home  score_prom_away  wins_home  wins_away  ties_home  \
 25732         1.454545         1.939394       11.0       17.0       10.0   
 25733         1.272727         2.060606       11.0       13.0        7.0   
 25734         1.666667         2.393939       16.0       22.0        7.0   
 25735         1.121212         0.969697        8.0        7.0        8.0   
 25736         1.636364         1.515152       15.0        8.0        7.0   
 
        ties_away  losses_home  losses_away  total_goals_home  \
 25732        4.0         12.0         12.0              48.0   
 25733       12.0         15.0          8.0              42.0   
 25734        7.0         10.0          4.0              55.0   
 25735        6.0         17.0         20.0              37.0   
 25736        8.0         11.0         17.0              54.0   
 
        total_goals_away  ...  points_home  points_away  streak_home  \
 25732              64.0  ...         43.0         55.0          4.0   


In [111]:
result = matches["result"]
matches = matches.drop(columns = ["Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk", "Date"])
matches.head()

columnas_categoricas = matches.select_dtypes(include=['object']).columns.tolist()

print("Columnas categóricas:")
print(columnas_categoricas)

Columnas categóricas:
[]


In [112]:
X = matches_train.to_numpy()
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
y_train = result_train

X_test = scaler.transform(matches_test.to_numpy())
y_test = result_test

In [113]:
X_train

array([[-2.21754941, -2.21178881, -1.25873609, ..., -1.20890707,
        -1.43218789, -1.38080256],
       [-2.21754941, -2.21178881, -1.25873609, ..., -1.20890707,
        -1.43218789, -1.38080256],
       [-2.21754941, -2.21178881, -1.25873609, ..., -1.20890707,
        -1.43218789, -1.38080256],
       ...,
       [ 0.68335525,  1.87853642,  1.81581105, ..., -0.20674266,
         0.21873252, -0.54256054],
       [-0.26603173, -0.55494821,  0.27853748, ..., -1.20890707,
        -0.60672769,  1.13392351],
       [ 0.63061153,  0.37702463,  1.62365185, ..., -0.20674266,
         0.21873252,  0.29568148]])

Modelo 

In [114]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50, # how to use learning rate scheduler
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
    )
    
clf.fit(X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=100, 
    batch_size=28, drop_last=False)

pred = clf.predict(X_test)

/Users/gustavofernandez/Desktop/Ing Civil/2024-1/IIC2985/iPre-resultados-deportivos/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.12649 | train_accuracy: 0.46934 | valid_accuracy: 0.47939 |  0:00:09s
epoch 1  | loss: 1.05425 | train_accuracy: 0.4714  | valid_accuracy: 0.48288 |  0:00:19s
epoch 2  | loss: 1.04225 | train_accuracy: 0.48049 | valid_accuracy: 0.49126 |  0:00:29s
epoch 3  | loss: 1.03217 | train_accuracy: 0.48896 | valid_accuracy: 0.49476 |  0:00:38s
epoch 4  | loss: 1.02985 | train_accuracy: 0.49304 | valid_accuracy: 0.49755 |  0:00:48s
epoch 5  | loss: 1.02589 | train_accuracy: 0.49425 | valid_accuracy: 0.50314 |  0:00:58s
epoch 6  | loss: 1.0256  | train_accuracy: 0.49401 | valid_accuracy: 0.50175 |  0:01:07s
epoch 7  | loss: 1.0227  | train_accuracy: 0.49572 | valid_accuracy: 0.50594 |  0:01:17s
epoch 8  | loss: 1.02306 | train_accuracy: 0.49569 | valid_accuracy: 0.50384 |  0:01:27s
epoch 9  | loss: 1.02152 | train_accuracy: 0.49522 | valid_accuracy: 0.50943 |  0:01:36s
epoch 10 | loss: 1.0214  | train_accuracy: 0.49592 | valid_accuracy: 0.51502 |  0:01:47s
epoch 11 | loss: 1.01

/Users/gustavofernandez/Desktop/Ing Civil/2024-1/IIC2985/iPre-resultados-deportivos/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
